In [1]:
import sys
import json
from collections import defaultdict, Counter
import itertools

from allennlp.models.archival import load_archive
from allennlp.predictors import Predictor
from allennlp.models.ensemble import Ensemble

from tqdm import tqdm
import numpy as np

import torch

In [2]:
import matplotlib.pyplot as plt

%matplotlib
%matplotlib inline  

Using matplotlib backend: agg


In [3]:
sys.path.append('../src/framework/models/')
sys.path.append('../src/framework/dataset_readers/')
sys.path.append('../src/framework/predictors/')
from deep_moji_model import DeepMojiModel
from deep_moji_reader import DeepMojiReader
# from src.framework.predictors

In [4]:
archive_model = load_archive('../src/allen_logs/deep_moji_balanced2/model.tar.gz')#, cuda_device=3)
# predictor_elmo = Predictor.from_archive(archive_elmo, 'nfh_classification')

In [5]:
archive_model

Archive(model=DeepMojiModel(
  (scorer): Sequential(
    (0): Linear(in_features=2304, out_features=300, bias=True)
    (1): Tanh()
    (2): Linear(in_features=300, out_features=2, bias=True)
  )
  (loss): CrossEntropyLoss()
), config=<allennlp.common.params.Params object at 0x7f5b2e0411d0>)

In [7]:
archive_model.model

DeepMojiModel(
  (scorer): Sequential(
    (0): Linear(in_features=2304, out_features=300, bias=True)
    (1): Tanh()
    (2): Linear(in_features=300, out_features=2, bias=True)
  )
  (loss): CrossEntropyLoss()
)

In [8]:
vec = torch.tensor(np.random.rand(1, 2304)).float()

In [9]:
archive_model.model(vec)

{'y_hat': tensor([1])}

In [10]:
module = archive_model.extract_module('scorer')

In [14]:
len(module)

3

In [15]:
x = vec
for layer in module[:2]:
    x = layer(x)
#     print(x)

In [16]:
x

tensor([[ 0.9787, -0.9997,  0.9825,  0.6857,  0.9304, -0.8528,  0.9981,  0.2299,
         -0.9999,  0.9967,  0.3081,  0.9989, -0.6227,  0.7911, -0.9767,  0.9969,
         -0.9004, -0.8723, -0.9890, -0.1890,  0.5829, -0.9551,  0.9999,  0.4626,
          0.4074, -0.9984,  0.9801,  0.9499,  0.9466, -0.9944, -0.5481,  1.0000,
         -0.6941,  0.9813, -0.8472, -1.0000,  0.9629,  0.9156,  0.9137, -0.0885,
          0.5921, -0.3396, -0.9464, -0.9992,  0.7965,  0.9531,  0.9889,  0.9954,
         -0.9517, -0.9913, -0.8844,  0.9999, -0.5206,  0.9956, -0.5383,  0.9875,
          0.6395, -0.9851, -0.9914,  0.9985,  0.9881,  0.7991,  0.9675,  0.1948,
          1.0000, -0.9966,  0.7548,  0.9868,  0.9999, -0.9081, -0.9952, -0.4961,
         -0.9280, -0.9879, -0.6180, -0.9677, -0.3289, -0.9575,  0.9973,  1.0000,
          0.4667, -0.9969, -0.7461, -1.0000, -0.9193,  0.5447, -0.9547, -0.9975,
         -0.9468, -0.9972,  0.6840,  0.9956,  1.0000,  0.9991, -0.9595, -0.8784,
          0.8830,  0.9958,  

In [30]:
def transform_vec(in_vec, model):
    module = model.extract_module('scorer')
    
    x = torch.tensor(in_vec).float()
    for layer in module[:2]:
        x = layer(x)
    return x

In [37]:
def read_data_file(input_file: str):
    vecs = np.load(input_file)
    return vecs

def calculate_vectors(in_dir, out_dir):
    for split in ['train', 'dev', 'test']:
        for vectors in ['pos_pos', 'pos_neg', 'neg_pos', 'neg_neg']:
            data = read_data_file(in_dir + '/' + split + '/' + vectors + '.npy')
            transformed_vec = transform_vec(data, archive_model)
            np.save(out_dir + '/' + split + '/' + vectors + '.npy', transformed_vec)

In [38]:
calculate_vectors('../data/emoji_sent_race/', '../data/emoji_sent_race_mlp')